<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [11]:

#!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon
!python3 -m spacy link en_core_web_lg en_lg
import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint
from nltk.tokenize import word_tokenize

nltk.download('punkt')

nlp = spacy.load("en_lg")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en_lg
You can now load the model via spacy.load('en_lg')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Language Analysis***

In [39]:

"""
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …



Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X's have influence on Y
X be Y's influence
Y cite X as influence
X was cited as influence by Y

influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y

influencing:
none

influenced:
Y be influenced by X
X have influenced Y
X influenced Y
Y's thinking was influenced by X
X influenced Y's work
X's work influenced Y



"""

#prints Dependencies and displacy visuals for a list of phrases
def get_info(sent_list):
  for sent in sent_list:
    print(sent)
    doc = nlp(sent)
    displacy.render(doc, style="dep", jupyter=True)  
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])


def process_influence(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    #se il chunk è  un propn o un noun allora spizzalo

def process_influenced(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    compounds = []
    for token in doc:
        #è un soggetto passivo, ex Mark was infuenced
        if token.dep_ == "nsubjpass" and token.head.text == "influenced" :
            influenced.append(token.text)

        #è un soggetto attivo, ex Jhon influenced
        if token.dep_ == "nsubj" and token.head.text == "influenced":
            influencers.append(token.text)
    
        #è un oggetto
        if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
            flag = 0
            tok = token.text
            while flag == 0:
                if token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                    token = token.head
                elif token.head.text == "influenced":
                    #see if it is passive or active
                    if "auxpass" in [x.dep_ for x in token.head.children]:
                        influencers.append(tok)
                    else:
                        influenced.append(tok)
                    flag = 1
                else:
                    flag = 1
        
        #è un genitivo
        if token.dep_ == "poss":
            tok = token.text
            token = token.head
            #di un soggetto
            if token.dep_== "nsubjpass" and token.head.text == "influenced" :
                influenced.append(tok)
            if token.dep_ == "nsubj" and token.head.text == "influenced":
                influencers.append(tok)
            #di un oggetto
            if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
                flag = 0
                while flag == 0:
                    if token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                        token = token.head
                    elif token.head.text == "influenced":
                        #see if it is passive or active
                        if "auxpass" in [x.dep_ for x in token.head.children]:
                            influencers.append(tok)
                        else:
                            influenced.append(tok)
                        flag = 1
                    else:
                        flag = 1

        #catch compounds
        if token.dep_ == "compound":
            compounds.append(token)
    
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers

def process_inspired(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    compounds = []
    for token in doc:
        #è un soggetto passivo, ex Mark was infuenced
        if token.dep_ == "nsubjpass" and token.head.text == "inspired" :
            influenced.append(token.text)

        #è un soggetto attivo, ex Jhon influenced
        if token.dep_ == "nsubj" and token.head.text == "inspired":
            influencers.append(token.text)
    
        #è un oggetto
        if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
            flag = 0
            tok = token.text
            while flag == 0:
                if token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                    token = token.head
                elif token.head.text == "inspired":
                    #see if it is passive or active
                    if "auxpass" in [x.dep_ for x in token.head.children]:
                        influencers.append(tok)
                    else:
                        influenced.append(tok)
                    flag = 1
                else:
                    flag = 1
        
        
        #è un genitivo
        if token.dep_ == "poss":
            tok = token.text
            token = token.head
            #di un soggetto
            if token.dep_== "nsubjpass" and token.head.text == "inspired" :
                influenced.append(tok)
            if token.dep_ == "nsubj" and token.head.text == "inspired":
                influencers.append(tok)
            #di un oggetto
            if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
                flag = 0
                while flag == 0:
                    if token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                        token = token.head
                    elif token.head.text == "inspired":
                        #see if it is passive or active
                        if "auxpass" in [x.dep_ for x in token.head.children]:
                            influencers.append(tok)
                        else:
                            influenced.append(tok)
                        flag = 1
                    else:
                        flag = 1

        #catch compounds
        if token.dep_ == "compound":
            compounds.append(token)

    #look for compound names
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers
    
phrases_influence=[["He was an influence of her",["her"],["He"]],
                   ["Jhon was an influence of Mark",["Mark"],["Jhon"]],
                   ["Milton was the main influence of Mallio and Tullio",["Milton"],["Mallio","Tullio"]],
                   ["Jhon's influence on Mark",["Mark"],["Jhon"]],
                   ["Markov's influence on Tito is the main reason of \"La Bella e la Bestia\", the main work of Tito's career. ",["Tito"],["Markov"]],
                   ["Jhon was Mark's influence",["Mark"],["Jhon"]],
                   ["At the university she was introduced to the writings of Aristotle and Plato, who would be her greatest influence and counter-influence respectively",["she"],["Aristotele", "Plato"]],
                   ["Jhon had influence on Mark's work",["Mark"],["Jhon"]],
                   ["Hegel had an important influence on all the different works of Einstein career",["Einstein"],["Hegel"]],
                   ["Mark cites Jhon as an influence",["Mark"],["Jhon"]],
                   ["Hegel cited Shopenauer as the main influence of his work",["Hegel"],["Shopenauer"]],
                   ["Jhon was cited by Mark as an influence",["Mark"],["Jhon"]],
                   ["Adler is cited as an influence on John Milton's testament",["Jhon", "Milton"],["Adler"]]]
phrases_influences=[["influences include him",[],["him"]],
                   ["influences include Jhon",[],["Jhon"]],
                   ["influences of his work include Jhon Milton and the author of  \"La Casa\", Rick Morty",[],["Jhon", "Milton","Rick","Morty"]],
                   ["Jhon and Adam were Mark's influences",["Mark"],["Jhon"]],
                   ["Aristotele, Anassagora and Pluto were Antichiola main influences",["Antichiola"],["Pluto","Aristotele","Anassagora"]],
                   ["Jhon and Adam where influences of Mark",["Mark"],["Jhon","Adam"]],
                   ["Macchiavelli and Dante Alighieri where the main influences of Boccaccio's Decameron",["Boccaccio"],["Macchiavelli","Dante", "Alighieri"]]]
phrases_influenced=[["He was influenced by him",["He"],["him"]],
                   ["Mark was influenced by Jhon",["Mark"],["Jhon"]],
                   ["Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts",["Adler"],["Immanuel", "Kant", "Friedrich", "Nietzsche", "Rudolf", "Virchow", "Jan", "Smuts"]],
                   ["Jhon has influenced Mark",["Mark"],["Jhon"]],
                   ["Macchiavelli with his work have deeply influenced the philosophies of Mathma Ghandi",["Mathma","Ghandi"],["Macchiavelli"]],
                   ["Jhon influenced Mark",["Mark"],["Jhon"]],
                   ["Anassagora influenced Pitagora on the ideas regarding geometry",["Pitagora"],["Anassagora"]],
                   ["Mark's work was influenced by Jhon",["Mark"],["Jhon"]],
                   ["Gandhi's \"Power to the Man\" was influenced by the medioeval thinker Mark Robben",["Ghandi"],["Mark", "Robben"]],
                   ["Jhon's work influenced Mark",["Mark"],["Jhon"]],
                   ["Marconi's work on the electromagnetic field influenced Einstein's ideas of photon's transmission",["Einstein"],["Marconi"]],
                   ["Jhon influenced Mark's work",["Mark"],["Jhon"]],
                   ["Pluto influenced the idea of time of Aristotele and Minos",["Aristotele", "Minos"],["Pluto"]]]
phrases_inspire=[
                    ["He seemed to inspire her",["her"],["He"]],
                    ["These revolutionizing ideas of Wang Yangming would later inspire prominent Japanese thinkers like Motoori Norinaga",["Mootori Norinaga"],["Wang Yangming"]],
                    ["John's work inspire Mark",["John"],["Mark"]]
                ]
phrases_inspiration=[
                        ["John was inspiration for Mark",["Mark"],["John"]],
                        ["Rozanov is the main source of inspiration for Dmitry Galkovsky",["Dmitry Galkovsky"],["Rozanov"]],
                        ["Jhon and Adam became inspiration for Mark",["Mark"],["John and Adam"]],
                        ["Jhon's work provided inspiration for Mark",["Mark"],["John"]],
                        ["He got the inspiration for this text from Schleiermacher ’ s Über die Religion ",["He"],["Schleiermacher"]],
                        ["Jhon's work was inspiration for Mark",["Mark"],["John"]],
                        ["While Murdoch 's thought is an inspiration for Conradi",["Conradi"],["Murdoch"]],
                        ["Jhon's work served as inspiration to Mark",["Mark"],["Jhon"]],
                        ["Lucian 's True Story inspired Cyrano de Bergerac , whose writings later served as inspiration for Jules Verne ",["Cyrano de Bergerac", "Jules Verne"],["Lucian"]],
                        ["Mark took inspiration from John",["Mark"],["John"]],
                        ["He also took inspiration from phenomenologist epistemology",["He"],["phenomenologist epistemology"]],
                        ["Mark drew inspiration from John", ["Mark"], ["John"]],
                        ["In particular , he drew inspiration from a Chinese Buddhist master named Tao-cho",["he"],["Tao-cho"]],
                        ["Mark provided inspiration to John",["John"],["Mark"]]
                    ]
phrases_inspired=[
                    ["He was inspired by him",["He"],["him"]],
                    ["Mark have inspired Jhon",["John"],["Mark"]],
                    ["Jhon had been inspired by Mark",["John"],["Mark"]],
                    ["In it , Petrarch claimed to have been inspired by Philip V of Macedon",["Petrarch"],["Philip V of Macedon"]],
                    ["Jhon's thinking was inspired by Mark",["John"],["Mark"]],
                    ["Newton 's work on infinite series was inspired by Simon Stevin 's decimals",["Newton"],["Simon Stevin"]],
                    ["Jhon's work was inspired by Marks",["John"],["Marks"]],
                    [ "Schiller was inspired by the play Julius of Tarent by Johann Anton Leisewitz .",["Schiller"],["Johann Anton Leisewitz"]],
                    [ "Mark was inspired by Jhon",["Mark"],["John"]],
                    ["As a youth , he was inspired by Mencius ’ proposition",["he"],["Mencius"]],
                    ["Jhon inspired Mark",["John"],["Mark"]],
                    [ "It also inspired him to take falsifiability as his criterion of demarcation between what is , and is not , genuinely scientific",["him"],["It"]],
                    ["Jhon inspired Mark's work",["Mark"],["John"]],
                    [ "Spinoza inspired the poet Shelley to write his essay",["Shelley"],["Spinoza"]]
                  ]


for phr in phrases_inspire:
  get_info([phr[0]])
  influenced, influencer = process_inspired(phr[0])
  print("______EVAL______")
  print("influenced = ",phr[1], "influencers", phr[2])
  print("influenced = ",influenced, "influencers", influencer)
  print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
  print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))



He seemed to inspire her


He nsubj seemed VERB []
seemed ROOT seemed VERB [He, inspire]
to aux inspire VERB []
inspire xcomp seemed VERB [to, her]
her dobj inspire VERB []
______EVAL______
influenced =  ['her'] influencers ['He']
influenced =  [] influencers []
0 on 1
0 on 1
These revolutionizing ideas of Wang Yangming would later inspire prominent Japanese thinkers like Motoori Norinaga


These det ideas NOUN []
revolutionizing amod ideas NOUN []
ideas nsubj inspire VERB [These, revolutionizing, of]
of prep ideas NOUN [Yangming]
Wang compound Yangming PROPN []
Yangming pobj of ADP [Wang]
would aux inspire VERB []
later advmod inspire VERB []
inspire ROOT inspire VERB [ideas, would, later, thinkers]
prominent amod thinkers NOUN []
Japanese amod thinkers NOUN []
thinkers dobj inspire VERB [prominent, Japanese, like]
like prep thinkers NOUN [Norinaga]
Motoori compound Norinaga PROPN []
Norinaga pobj like SCONJ [Motoori]
______EVAL______
influenced =  ['Mootori Norinaga'] influencers ['Wang Yangming']
influenced =  [] influencers []
0 on 1
0 on 1
John's work inspire Mark


John poss work NOUN ['s]
's case John PROPN []
work nsubj inspire VERB [John]
inspire ROOT inspire VERB [work, Mark]
Mark dobj inspire VERB []
______EVAL______
influenced =  ['John'] influencers ['Mark']
influenced =  [] influencers []
0 on 1
0 on 1


#***Import JSON articles***


In [0]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            {
                "0": "name_of_someone_philosopher_influenced_by"
            }
        ]
    "table_influences":
        [
            {
                "0": "name_of_someone_philosopher_influences"
            }
        ]
}
"""
#open the file
with open('/content/drive/My Drive/folder/result_1.json') as f:
  jsonlist = json.load(f)

# ***Text Preprocessing***

In [0]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, -ing etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""

"""
import time
start = time.time()
counter_influence = []
counter_influenced = []
counter_influences = []
counter_influencing = []
counter_inspired = []
counter_inspire = []
counter_inspiration = []
for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing", "inspired", "inspire", "inspiration"]


    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if "influence" in word_list:
            counter_influence.append(word_list)
        if "influencing" in word_list:
            counter_influencing.append(word_list)
        if "influenced" in word_list:
            counter_influenced.append(word_list)
        if "influences" in word_list:
            counter_influences.append(word_list)
        if "inspired" in word_list:
            counter_inspired.append(word_list)
        if "inspire" in word_list:
            counter_inspire.append(word_list)
        if "inspiration" in word_list:
            counter_inspiration.append(word_list)
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list
print(time.time() - start)

"""
"""
influencing pisciato
inspire pisciato

influence only used as a name
influences only used as a name, plural of influences
inspiration only used as a name, similar to influence
influenced used as verb
inspired used like influenced

X's can be changed with his
Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X be Y's influence

influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y

influencing:
none

influenced:
Y be influenced by X
X have influenced Y
Y's thinking was influenced by X
X influenced Y
"""




print(len(counter_influence), "influence",
      len(counter_influences), "influences",
      len(counter_influencing), "influencing",
      len(counter_influenced), "influenced",
      len(counter_inspired), "inspired",
      len(counter_inspire), "inspire",
      len(counter_inspiration), "inspiration")
for wl in counter_influence:
    print(' '.join(word for word in wl))

# ***Processing Giuliano(Spacy Dependencies)***

In [40]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" : "philosopher_name",
        "pageid" : id,
        "article" : 
            [
                "frase contenente influence",

                "frase contenente influence" 
            ]

        "rich_article" :
            {
                index: 
                {
                    "influenced" :
                        [
                            "list of influensed chunks"
                        ],
                    "Influencers" :
                        [
                            "list of influensers chunks"
                        ],
                    "phrase" : "frase contenente influence"
                }
            } 


                "frase contenente influence" = "Jhon Milton influenced Ghandi and Hannibal Bures' "Power to the Man""
                [
                    list of influencers = ["Jhon", "Milton"] --> "Jhon Milton"
                ],
                [
                    list of influenced = ["Ghandi", "Hannibal", "Bures", "Power", "Man"] --> "Mathma Ghandi", "Hannibal Bures"
                ]
                "him" --> nome del philosopher

"""



"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
"""
import re
print(jsonlist_no_empty[20]["article"][0])
tokens = [i for i in jsonlist_no_empty[20]["article"][0] if i.isupper() ]
print(tokens)
"""

for phil in jsonlist_no_empty:
    display_info(phil["article"])




NameError: ignored

# ***Processing Mattia(Regex)***

In [0]:
"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
#pprint.pprint(jsonlist)
stringone = 'His philosophy is mainly influenced by such thinkers as Nietzsche , Epicurus , the Cynic and Cyrenaic schools , as well as French materialism .'
relations = re.findall(r'[A-Z][a-z]+[\s]', stringone)
influence = re.findall(r'[iI]nfluenced[\s]', stringone)
dio = re.findall(r'[A-Z][a-z]+[\s].*[iI]nfluenced[\s].*[A-Za-z]+[\s]', stringone)
#print(re.split('[iI]nfluenced[\s]by', stringone ,2)[1])
left = re.split('[iI]nfluenced[\s]by', stringone ,2)[0]
#left = re.findall(r'[A-Z][a-z]+[\s].*[iI]nfluenced[\s]', stringone)
right = re.split('[iI]nfluenced[\s]by', stringone ,2)[1]
#right = re.findall(r'[iI]nfluenced by[\s].*', stringone)
influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
"""
print(stringone)
print('Left: ' + str(left))
print('Right: ' + str(right))
print('Influencers: ' + str(influencers))
print('Influenced: ' + str(influenced))
"""



jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]

for pippo in jsonlist_no_empty:
  #print(len(pippo['article']))
  #print(pippo.keys())
  pippo['influencers'] = []
  pippo['influenced'] = []
  pippo['rich_article'] = {}
  for i in range(len(pippo['article'])):
    left = []
    right = []
    influencers = []
    influenced = []

    #print('Sentence: ' + pippo['article'][i])
    try:
      left = re.split('have[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
      right = re.split('have[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
      influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
      influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
    except:
      try:
        left = re.split('was[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
        right = re.split('was[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
        influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
        influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
      except:
        try:
          left = re.split('[iI]nfluenced[\s]by[\s]', pippo['article'][i] ,2)[0]
          right = re.split('[iI]nfluenced[\s]by[\s]', pippo['article'][i] ,2)[1]
          influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
          influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
        except:
          try:
            left = re.split('has[\s][iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
            right = re.split('has[\s][iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
            influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
            influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
          except:
            try:
              left = re.split('[iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
              right = re.split('[iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
              influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
              influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
            except:
              try:
                left = re.split('[iI]nfluence[\s].*on[\s]', pippo['article'][i] ,2)[0]
                right = re.split('[iI]nfluence[\s].*on[\s]', pippo['article'][i] ,2)[1]
                influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
                influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
              except:
                try:
                  left = re.split('[iI]nfluence[\s].*of[\s]', pippo['article'][i] ,2)[0]
                  right = re.split('[iI]nfluence[\s].*of[\s]', pippo['article'][i] ,2)[1]
                  influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
                  influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
                except:
                  influencers = []
                  influenced = []
    pippo['rich_article'][i] = {}
    pippo['rich_article'][i]['phrase'] = pippo['article'][i]
    pippo['rich_article'][i]['influenced'] = influenced
    pippo['rich_article'][i]['influencers'] = influencers

    pippo['influencers'].append(influencers)
    pippo['influenced'].append(influenced)
    #print('Influencers: ' + str(influencers))
    #print('Influenced: ' + str(influenced))

for pippo in jsonlist_no_empty:
  pippo['influenced'] = [item for sublist in pippo['influenced'] for item in sublist]
  pippo['influencers'] = [item for sublist in pippo['influencers'] for item in sublist]


  pprint.pprint(pippo)

# ***Processing Luigi(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""